In [1]:
import os
%pwd

'c:\\Users\\Prarthana\\Desktop\\projects\\noise_reduction_autoencoder\\research'

In [2]:
os.chdir('../')
%pwd

'c:\\Users\\Prarthana\\Desktop\\projects\\noise_reduction_autoencoder'

In [3]:
#update entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    model_dir: Path
    image_size: int
    epochs: int

In [4]:
#update configurationmanager
from src import logger
from src.utils.common import read_yaml, create_directories
#from src.entity.config_entity import DataIngestionConfig, DataSplittingConfig, DataPreprocessingConfig, ModelTrainingConfig

class ConfigurationManager:
    def __init__(self, config_filepath="config/config.yaml", params_filepath="params.yaml"):
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        create_directories([self.config.artifacts_root])

    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        create_directories([config.model_dir])
        model_training_config = ModelTrainingConfig(
            model_dir=config.model_dir,
            image_size=config.image_size,
            epochs=config.epochs
        )
        return model_training_config


In [6]:
#update components
import tensorflow as tf
from pathlib import Path
from src import logger
#from src.entity.config_entity import ModelTrainingConfig

class ModelTraining:
    def __init__(self, config: ModelTrainingConfig, datasets: dict):
        self.config = config
        self.datasets = datasets
        self.model = None

    def build_autoencoder(self):
        """Build a convolutional autoencoder for denoising."""
        input_shape = (self.config.image_size, self.config.image_size, 1)  # Grayscale images
        
        # Encoder
        inputs = tf.keras.Input(shape=input_shape)
        x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
        x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
        x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
        x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
        encoded = tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)

        # Decoder
        x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(encoded)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
        x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
        x = tf.keras.layers.UpSampling2D((2, 2))(x)
        decoded = tf.keras.layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

        # Autoencoder model
        self.model = tf.keras.Model(inputs, decoded)
        self.model.compile(optimizer="adam", loss="mean_squared_error")
        self.model.summary(print_fn=lambda x: logger.info(x))
        
        return self.model

    def train(self):
        """Train the autoencoder using the preprocessed datasets."""
        try:
            logger.info("Starting model training")
            
            # Build the model
            self.build_autoencoder()
            
            # Train the model
            history = self.model.fit(
                self.datasets["train"],
                validation_data=self.datasets["val"],
                epochs=self.config.epochs,
                callbacks=[
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss",
                        patience=5,
                        restore_best_weights=True
                    ),
                    tf.keras.callbacks.ModelCheckpoint(
                        str(self.config.model_dir / "autoencoder_best.h5"),
                        monitor="val_loss",
                        save_best_only=True
                    )
                ]
            )
            
            # Evaluate on test set
            test_loss = self.model.evaluate(self.datasets["test"])
            logger.info(f"Test loss (MSE): {test_loss}")
            
            # Save the final model
            self.model.save(self.config.model_dir / "autoencoder_final.h5")
            logger.info(f"Model saved to {self.config.model_dir / 'autoencoder_final.h5'}")
            
            logger.info("Model training completed")
            return history
        
        except Exception as e:
            logger.error(f"Model training failed: {e}", exc_info=True)
            raise e

In [10]:
#update pipeline

try:
    # Initialize and train the model
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = ModelTraining(config=model_training_config, datasets=self.datasets)
    history = model_training.train()
except Exception as e:
    raise e

[2025-04-23 23:03:52,053: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-04-23 23:03:52,058: INFO: common: YAML file: params.yaml loaded successfully]
[2025-04-23 23:03:52,060: INFO: common: Created directory at: artifacts]
[2025-04-23 23:03:52,062: INFO: common: Created directory at: artifacts/models]


NameError: name 'self' is not defined

In [ ]:
#research

In [ ]:
import matplotlib.pyplot as plt
from src.pipeline.stage_05_model_training import ModelTrainingPipeline

obj = ModelTrainingPipeline()
history = obj.main()

plt.figure(figsize=(8, 6))
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Training and Validation Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Mean Squared Error")
plt.legend()
plt.grid(True)
plt.savefig("docs/plots/training_loss.png")
plt.close()
print("Loss plot saved to docs/plots/training_loss.png")

[2025-04-23 23:38:58,937: INFO: common: YAML file: c:\Users\Prarthana\Desktop\projects\noise_reduction_autoencoder\config\config.yaml loaded successfully]
[2025-04-23 23:38:58,940: INFO: common: YAML file: c:\Users\Prarthana\Desktop\projects\noise_reduction_autoencoder\params.yaml loaded successfully]
[2025-04-23 23:38:58,942: INFO: common: Created directory at: artifacts]
[2025-04-23 23:38:58,942: INFO: data_preprocessing: Starting data preprocessing]
[2025-04-23 23:38:58,953: INFO: data_preprocessing: train set: 436 clean images, 436 noisy images]
[2025-04-23 23:38:59,052: INFO: data_preprocessing: val set: 93 clean images, 93 noisy images]
[2025-04-23 23:38:59,068: INFO: data_preprocessing: test set: 95 clean images, 95 noisy images]
[2025-04-23 23:38:59,084: INFO: data_preprocessing: Data preprocessing completed]
[2025-04-23 23:38:59,086: INFO: common: Created directory at: artifacts/models]
[2025-04-23 23:38:59,087: INFO: model_training: Starting model training]


[2025-04-23 23:38:59,161: INFO: model_training: Model: "functional"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├───────

Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 512ms/step - loss: 0.0459

[2025-04-23 23:39:09,127: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


14/14 ━━━━━━━━━━━━━━━━━━━━ 10s 607ms/step - loss: 0.0451 - val_loss: 0.0149
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 844ms/step - loss: 0.0115

...

14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 771ms/step - loss: 0.0022 - val_loss: 0.0021
Epoch 19/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 687ms/step - loss: 0.0022

[2025-04-23 23:42:48,629: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 765ms/step - loss: 0.0022 - val_loss: 0.0021
Epoch 20/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 778ms/step - loss: 0.0022 - val_loss: 0.0033
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 249ms/step - loss: 0.0021


[2025-04-23 23:43:01,933: INFO: model_training: Test loss (MSE): 0.0020507618319243193]
[2025-04-23 23:43:01,936: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
[2025-04-23 23:43:02,051: INFO: model_training: Model saved to artifacts\models\autoencoder_final.h5]
[2025-04-23 23:43:02,053: INFO: model_training: Model training completed]


Loss plot saved to artifacts/plots/training_loss.png
